In [1]:
import sys
import os
os.environ['USE_PYGEOS'] = '0'
import gc
import collections

import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import geopandas as gpd

import numpy as np
import xarray as xr
# from xeofs.xarray import EOF
import rioxarray

import scipy
from scipy import signal
import cftime
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd

In [2]:
# File path directories

inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'
DIR_external = 'data/external/'
DIR_interim = 'data/interim/'

# DATASET FILEPATHS
# Basal melt observations from Paolo 2023
DIR_basalMeltObs = 'data/external/Paolo2023/'
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'

# INTERIM GENERATED FILEPATHS
DIR_basalMeltObs_Interim = 'data/interim/Paolo2023/iceShelves_dedraft/'
DIR_SORRMv21_Interim = 'data/interim/SORRMv2.1.ISMF/iceShelves_dedraft/'

# DATA FILENAMES
FILE_basalMeltObs = 'ANT_G1920V01_IceShelfMelt.nc'
FILE_MeltDraftObs = 'ANT_G1920V01_IceShelfMeltDraft.nc'
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'


FILE_iceShelvesShape = 'iceShelves.geojson'

In [3]:
MELTDRAFT_OBS = xr.open_dataset(inDirName+DIR_basalMeltObs+FILE_MeltDraftObs)
SORRMv21 = xr.open_dataset(inDirName+DIR_SORRMv21+FILE_SORRMv21)

ICESHELVES_MASK = gpd.read_file(inDirName+DIR_external+FILE_iceShelvesShape)
icems = ICESHELVES_MASK.to_crs({'init': 'epsg:3031'});
crs = ccrs.SouthPolarStereo();

/Users/smurugan9/opt/anaconda3/envs/aislens/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
sorrmv21_ts = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.mean(['x','y'])

In [37]:
"""
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(sorrmv21_ts, period=12)
ax = plt.figure(figsize=(25,8))
sorrmv21_ts.plot(label='SORRMv2.1 FreshwaterFlux (spatially aggregated)')
plt.plot(sorrmv21_ts.Time,result.trend, label='Annual Trend')
plt.xlabel('Time (Years)')
plt.title('Cumulative Freshwater Flux (AIS)')
plt.legend()
plt.savefig(inDirName+'reports/interim-fig/SORRMv2.1.ISMF/SORRMv2.1.ISMF.TimeSeries.png')
"""

In [19]:
SORRMv21.rio.write_crs("epsg:3031",inplace=True);
#h = SORRMv21.timeMonthly_avg_ssh
h_mean = SORRMv21.timeMonthly_avg_ssh.mean('Time')

In [8]:
MELTDRAFT_OBS.rio.write_crs("epsg:3031",inplace=True);
h = MELTDRAFT_OBS.draft
h_mean = h.mean('time')

In [4]:
def clip_data(DATA, iceShelfMask_index):
    masked_data = DATA.rio.clip(icems.loc[[iceShelfMask_index],'geometry'].apply(mapping), icems.crs, drop=False)
    return masked_data

In [ ]:
for i in range(6,32):
    print('extracting data for catchment {}'.format(icems.name.values[i]))
    mlt = xr.DataArray(np.empty(SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.shape),
                       dims = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.dims,
                       coords = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.coords,
                       attrs = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.attrs)
    for j in range(SORRMv21.Time.shape[0]):
        mlt[j] = clip_data(SORRMv21.timeMonthly_avg_landIceFreshwaterFlux[j], i)
    mlt_mean = mlt.mean('Time')
    # Dedraft: Linear Regression with SSH over chosen basin
    print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
    mlt_rgrs = xr_linregress(h, mlt_mean, dim='Time') # h = independent variable
    mlt_prd = mlt_rgrs.slope*h_mean + mlt_rgrs.intercept
    #flx_ddrft = flx - flx_prd
    mlt_prd.to_netcdf(inDirName+DIR_SORRMv21_Interim+'{}_rgrs.nc'.format(icems.name.values[i]))
    print('{} file saved'.format(icems.name.values[i]))
    del mlt, mlt_mean, mlt_rgrs, mlt_prd
    print('deleted interim variables')
    gc.collect()

extracting data for catchment IMBIE1


In [23]:
icems.loc[[32],'name']

32    IMBIE27
Name: name, dtype: object

In [24]:
for i in range(32,33):
    print('extracting data for catchment {}'.format(icems.name.values[i]))
    mlt = MELTDRAFT_OBS.melt.rio.clip(icems.loc[[i],'geometry'].apply(mapping),icems.crs,drop=False)
    mlt_mean = mlt.mean('time')
    #h = ds.timeMonthly_avg_ssh
    #h_mean = h.mean('time')
    # Dedraft: Linear Regression with SSH over chosen basin
    print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
    mlt_rgrs = xr_linregress(h, mlt_mean, dim='time') # h = independent variable
    mlt_rgrs.to_netcdf(inDirName+DIR_basalMeltObs_Interim+'{}_rgrs_vals.nc'.format(icems.name.values[i]))
    #flx_ddrft = flx - flx_prd
    mlt_prd.to_netcdf(inDirName+DIR_basalMeltObs_Interim+'{}_rgrs.nc'.format(icems.name.values[i]))
    #print('{} file saved'.format(icems.name.values[i]))
    del mlt, mlt_mean, mlt_rgrs, mlt_prd
    print('finished')
    #gc.collect()

extracting data for catchment IMBIE27
calculating linear regression for catchment IMBIE27
finished


In [27]:
# Merge regression files of individual ice shelves

ds_np = np.empty(MELTDRAFT_OBS.melt[0].shape)
ds_np[:] = np.nan

ICESHELF_RGRS = xr.DataArray(ds_np, 
                             coords=MELTDRAFT_OBS.melt[0].coords, 
                             dims = MELTDRAFT_OBS.melt[0].dims, 
                             attrs=MELTDRAFT_OBS.melt.attrs)
#iceshelves_rgrs = xr.Dataset(data_vars=dict(timeMonthly_avg_landIceFreshwaterFlux=(iceshelves_rgrs_array)), coords=data.coords, attrs=data.timeMonthly_avg_landIceFreshwaterFlux.attrs)
ICESHELF_RGRS = xr.Dataset(data_vars=dict(melt=(ICESHELF_RGRS)))

for i in range(6,33):
    iceshelves_rgrs_catchment = xr.open_dataset(inDirName+DIR_basalMeltObs_Interim+'{}_rgrs.nc'.format(icems.name.values[i]))
    iceshelves_rgrs_catchment['melt'] = iceshelves_rgrs_catchment['__xarray_dataarray_variable__']
    iceshelves_rgrs_catchment = iceshelves_rgrs_catchment.drop(['__xarray_dataarray_variable__'])
    ICESHELF_RGRS = xr.merge([ICESHELF_RGRS, iceshelves_rgrs_catchment], compat='no_conflicts')

ICESHELF_RGRS.to_netcdf(inDirName+DIR_basalMeltObs_Interim+"ICESHELVES_AIS_RGRS.nc")

In [32]:
# Merge regression files of individual ice shelves

ds_np = np.empty(SORRMv21.timeMonthly_avg_landIceFreshwaterFlux[0].shape)
ds_np[:] = np.nan

ICESHELF_RGRS = xr.DataArray(ds_np, 
                             coords=SORRMv21.timeMonthly_avg_landIceFreshwaterFlux[0].coords, 
                             dims = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux[0].dims, 
                             attrs=SORRMv21.timeMonthly_avg_landIceFreshwaterFlux.attrs)
#iceshelves_rgrs = xr.Dataset(data_vars=dict(timeMonthly_avg_landIceFreshwaterFlux=(iceshelves_rgrs_array)), coords=data.coords, attrs=data.timeMonthly_avg_landIceFreshwaterFlux.attrs)
ICESHELF_RGRS = xr.Dataset(data_vars=dict(timeMonthly_avg_landIceFreshwaterFlux=(ICESHELF_RGRS)))

for i in range(6,33):
    iceshelves_rgrs_catchment = xr.open_dataset(inDirName+DIR_SORRMv21_Interim+'{}_rgrs.nc'.format(icems.name.values[i]))
    iceshelves_rgrs_catchment['timeMonthly_avg_landIceFreshwaterFlux'] = iceshelves_rgrs_catchment['__xarray_dataarray_variable__']
    iceshelves_rgrs_catchment = iceshelves_rgrs_catchment.drop(['__xarray_dataarray_variable__'])
    ICESHELF_RGRS = xr.merge([ICESHELF_RGRS, iceshelves_rgrs_catchment], compat='no_conflicts')

ICESHELF_RGRS.to_netcdf(inDirName+DIR_SORRMv21_Interim+"ICESHELVES_AIS_RGRS.nc")

# Dedraft data

DEDRAFTED_SORRM = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux - ICESHELF_RGRS.timeMonthly_avg_landIceFreshwaterFlux
DEDRAFTED_SORRM = xr.Dataset(data_vars=dict(timeMonthly_avg_landIceFreshwaterFlux=(DEDRAFTED_SORRM)))
DEDRAFTED_SORRM.to_netcdf(inDirName+DIR_interim+"SORRMv2.1./DEDRAFTED_SORRM.nc")

In [33]:
# Dedraft data

DEDRAFTED_MELT_OBS = MELTDRAFT_OBS.melt - ICESHELF_RGRS.melt
DEDRAFTED_MELT_OBS = xr.Dataset(data_vars=dict(melt=(DEDRAFTED_MELT_OBS)))
DEDRAFTED_MELT_OBS.to_netcdf(inDirName+DIR_interim+"Paolo2023/DEDRAFTED_MELT_OBS.nc")

In [47]:
MELTDRAFT_OBS.melt.attrs

{'units': 'm of ice per year',
 'standard_name': 'ice shelf basal melt rate',
 'density': 917.0,
 'density_units': 'kg m^-3',
 'grid_mapping': 'mapping'}

In [4]:
# m / yr
sec_per_yr = 60*60*24*365
rho_ice = 917.0 # kg/m^3
rho_fw = 1000.0 
#MELTDRAFT_MELTOBS = MELTDRAFT_OBS.melt*rho_fw/sec_per_yr

In [5]:
DEDRAFTED_MELT_OBS = xr.open_dataset(inDirName+DIR_interim+"Paolo2023/DEDRAFTED_MELT_OBS.nc")
DEDRAFTED_MELT = DEDRAFTED_MELT_OBS.melt*rho_fw/sec_per_yr
DEDRAFTED_MELT.rio.write_crs("epsg:3031",inplace=True);

In [6]:
ds_np = np.empty(len(range(6,33)))
ds_np[:] = np.nan

for i,isnum in enumerate(range(6,33)):
    ICESHELF_MELT = DEDRAFTED_MELT.rio.clip(icems.loc[[isnum],'geometry'].apply(mapping),icems.crs,drop=False)
    ds_np[i] = ICESHELF_MELT.mean()
    print("calculated for {}".format(icems.name.values[isnum]))
    #iceshelves_rgrs_catchment = iceshelves_rgrs_catchment.drop(['__xarray_dataarray_variable__'])
    #ICESHELF_RGRS = xr.merge([ICESHELF_RGRS, iceshelves_rgrs_catchment], compat='no_conflicts')
    del ICESHELF_MELT
    gc.collect()


calculated for IMBIE1
calculated for IMBIE2
calculated for IMBIE3
calculated for IMBIE4
calculated for IMBIE5
calculated for IMBIE6
calculated for IMBIE7
calculated for IMBIE8
calculated for IMBIE9
calculated for IMBIE10
calculated for IMBIE11
calculated for IMBIE12
calculated for IMBIE13
calculated for IMBIE14
calculated for IMBIE15
calculated for IMBIE16
calculated for IMBIE17
calculated for IMBIE18
calculated for IMBIE19
calculated for IMBIE20
calculated for IMBIE21
calculated for IMBIE22
calculated for IMBIE23
calculated for IMBIE24
calculated for IMBIE25
calculated for IMBIE26
calculated for IMBIE27


TypeError: _save_dispatcher() missing 1 required positional argument: 'arr'

In [7]:
np.save(inDirName+DIR_interim+"Paolo2023/imbie_iceshelf_mean_fluxrates.npy", ds_np)

In [9]:
plt.scatter(range(6,33),ds_np)

In [10]:
# IMBIE ice shelves = 6 to 32
for i,isnum in enumerate(range(6,33)):
    print(i,isnum)

0 6
1 7
2 8
3 9
4 10
5 11
6 12
7 13
8 14
9 15
10 16
11 17
12 18
13 19
14 20
15 21
16 22
17 23
18 24
19 25
20 26
21 27
22 28
23 29
24 30
25 31
26 32


In [ ]:
# Save linear regression interim files for MELT_OBS and SORRMv21

In [ ]:
# Plot dedrafted melt for MELT_OBS and SORRMv21 across ice shelves.
# To save time, can use IMBIE definitions of ice shelves at first - 
# these are larger catchments with mostly similar behavior as the 
# refined catchment definitions of individual ice shelves

In [ ]:
# Compare PAOLO23 to SORRMv21
# Spatiotemporal means in a scatterplot

In [75]:
MELTDRAFT_OBS.melt.attrs

{'units': 'm of ice per year',
 'standard_name': 'ice shelf basal melt rate',
 'density': 917.0,
 'density_units': 'kg m^-3',
 'grid_mapping': 'mapping'}

In [49]:
# Refer Paolo 2023, or captoolkit
# Lines 260-261 in 
# https://github.com/nasa-jpl/captoolkit/blob/be9aa159f2eec912e55d7b031f16ac026c8bd045/captoolkit/cubethick.py#L260C66-L260C66

# D = (1 - (rho_ocean - rho_ice) / rho_ocean) H

# H_draft = H_freeb * ((rho_ocean / (rho_ocean - rho_ice)) - 1)
# H = H_freeb * rho_ocean / (rho_ocean - rho_ice)

H = melt.thickness
rho_ocean = 1028.0
rho_ice = 917.0
H_draft = (H*(rho_ocean-rho_ice)/rho_ocean)*((rho_ocean / (rho_ocean - rho_ice)) - 1)
melt['draft'] = H_draft


# xarray raises an issue when saving the modified dataset with just the melt and draft variables, 
# delete grid_mapping attribute to proceed with writing the file
# del ds.melt.attrs['grid_mapping']

ds1 = melt.melt
ds2 = melt.draft
ds = xr.merge([ds1,ds2])
ds.to_netcdf(inDirName+DIR_basalMeltObs+'ANT_G1920V01_IceShelfMeltDraft.nc')

In [76]:
mlt = melt.melt.rio.clip(icems.loc[[i],'geometry'].apply(mapping),icems.crs,drop=False)
mlt_mean = mlt.mean('time')
#h = ds.timeMonthly_avg_ssh
#h_mean = h.mean('time')
# Dedraft: Linear Regression with SSH over chosen basin
print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
mlt_rgrs = xr_linregress(h, mlt_mean, dim='time') # h = independent variable
mlt_prd = mlt_rgrs.slope*h_mean + mlt_rgrs.intercept
#flx_ddrft = flx - flx_prd

calculating linear regression for catchment IMBIE1


extracting data for catchment IMBIE1
calculating linear regression for catchment IMBIE1
IMBIE1 file saved
deleted interim variables
extracting data for catchment IMBIE2
calculating linear regression for catchment IMBIE2
IMBIE2 file saved


NameError: name 'ds' is not defined

In [85]:
# Scatterplots of slope of ice shelves in Paolo vs. SORRM
# Scatterplots of ice shelf spatiotemporal means in dedrafted version

'data/interim/Paolo2023/iceShelves_dedraft/'